The need to interactively tune the fiducial marker detection and check its output means that drift correction should not be performed in batch. However, the drift correction process can be made simpler by using a notebook that assists in finding the files and performing the drift correction. This notebook serves as the template for this step.

Note: Typically, this step would be performed before the actual batch processing in tutorial 2. Since tutorial 2 was easier, though, I thought it would be best to explain it before this one.

### Import the software libraries

In [1]:
%pylab
import DataSTORM.processors as proc
import pandas as pd
from pathlib import Path

Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


# Outline of steps
1. Search the parent directory for all localization files and a build a list of such files
2. Define the drift correction processor
3. Open a file from the list
4. Perform the drift correction on the file
  * If the correction was not good, go back to step 2
4. Save the results
5. Repeat from step 1

# Search the parent directory and make a list of localization files
We will start by searching a parent directory and its subdirectories for all the localization files, i.e. those files that end in `locResults.dat`.

In [2]:
parentDirectory   = Path('../test-data/Centrioles/')
localizationFiles = parentDirectory.glob('**/*locResults.dat')
locResultFiles    = sorted(localizationFiles)

# How many files are there?
print(len(locResultFiles))

2


`locResultFiles` is an array of files ending in `locResults.dat` within the parent directory `../test-data/Centrioles/` and its subdirectories.

In [4]:
locResultFiles[0] # In Python, array indexes start at 0

PosixPath('../test-data/Centrioles/FOV_1_noPB_1500mW_10ms_1/FOV_1_noPB_1500mW_10ms_1_MMStack_locResults.dat')

In [5]:
locResultFiles[1]

PosixPath('../test-data/Centrioles/FOV_7_noPB_1500mW_10ms_1/FOV_7_noPB_1500mW_10ms_1_MMStack_locResults.dat')

# Define the drift correction processor
This part is the same as in Tutorial 1. At the end of processing for each file, return to this code block and start running the blocks again.

In [7]:
# Create the FiducialDriftCorrect processor and set its properties.
dc = proc.FiducialDriftCorrect(minFracFiducialLength = 0.5,
                               neighborRadius        = 500,
                               smoothingWindowSize   = 2000,
                               smoothingFilterSize   = 500,
                               interactiveSearch     = True,
                               noLinking             = True)

clean = proc.CleanUp()

# Open a file from the list
Now, we're going to open the first file in this list, locResultFiles[0]. When we are all done with this file, we will come back to here and increase the index by one (to locResultFiles[1], and so on...).

In [8]:
# Set the input and output files
currentFile = locResultFiles[0] # Increment this when done dedrifting a dataset
outputFile  = currentFile.parent / Path(currentFile.stem + '_DC' + currentFile.suffix)

# Open a file and clean it up
with open(str(currentFile), 'r') as file:
    df = pd.read_csv(file)
    
df = clean(df)